In [ ]:
import numpy as np

# Fonctions

In [17]:
def ReLu(x):
    # if (x < 0):
    #     return 0
    # return x
    return np.maximum(0,x)

def softmax(x):
    sum = 0
    for i in range(len(x)):
        sum += np.exp(x[i])
    return np.exp(x) / sum


# def Max3x3(M): // a adapter car pour le moment j'utilise  np.max ca fait le taff
#     max = M[0][0]
#     for i in range(3):
#         for j in range(3):
#             if M[i][j] > max:
#                 max = M[i][j]
#     return max

def MaxPool(M):
    y,x,num_channels = M.shape
    i = 0
    j = 0
    output_height = (y - 3) // 2 + 1
    output_width = (x - 3) // 2 + 1
    print(f"output height: {output_height}, output width: {output_width}")
    output = np.zeros((output_height, output_width, num_channels))

    for c in range(num_channels):
        for i in range(0, output_height):
            for j in range(0, output_width):
                start_i, start_j = i * 2, j * 2
                region = M[start_i:start_i + 3, start_j:start_j + 3, c] #j'ai enlever output x et y car pas besoin et re for pour eviter de remettre à 0
                output[i, j, c] = np.max(region)
    return output

def MaxPool_v2(M):
    y,x,num_channels = M.shape
    i = 0
    j = 0
    output_height = y // 2
    output_width = x // 2
    output = np.zeros((output_height, output_width, num_channels))
    for c in range(num_channels):
        for i in range(0, output_height):
            for j in range(0, output_width):
                start_i, start_j = i * 2, j * 2
                if (start_i + 3 > y):
                    if(start_j + 3 > x):
                        region = M[start_i:y, start_j:start_j + 3, c]
                    else:
                        region = M[start_i:y, start_j:x, c]
                elif (start_j + 3 > x):
                    region = M[start_i:start_i + 3, start_j:x, c]
                else:
                    region = M[start_i:start_i + 3, start_j:start_j + 3, c] #j'ai enlever output x et y car pas besoin et re for pour eviter de remettre à 0
                output[i, j, c] = np.max(region)
    return output

def convolution(image, Ks, biais):
    image_height, image_width, image_channels = image.shape
    num_filters, K_height, K_width, _ = Ks.shape
    output_height = image_height
    output_width = image_width
    output = np.zeros((output_height, output_width, num_filters))
    padded_image = np.pad(image, ((2, 2), (2, 2), (0, 0)), mode='constant') #j'ai vu qu'il fallait faire du padding pour conserver les dimensions de notre frame 3D 
   
    for f in range(num_filters):
        conv_sum = np.zeros((output_height, output_width))
        for c in range(image_channels):  
            for i in range(output_height):
                for j in range(output_width):
                    region = padded_image[i:i+K_height, j:j+K_width, c]
                    conv_sum[i, j] += np.sum(region * Ks[f, :, :, c])
        output[:, :, f] = ReLu(conv_sum + biais[f])
   
    return output

def reshape(M):
    height, width, channels = M.shape
    output = M
    return output.reshape(height*width*channels) # On garde comme ça pour l'instant mais à changer pour le matériel

def FCP(M, weights, bias): # Fully Connected Perceptron
    '''
    M : (180,)
    weights : (180, 10)
    bias : (10,)
    output : (10,)
    '''
    M = M.T
    output = softmax(M.dot(weights) + bias)
    return output

# Modèle Complet

In [ ]:
input_image = np.random.randint(0, 1, size=(24, 24, 3))
k1 = np.random.rand(64, 3, 3, 3)
biais1 = np.random.rand(64)
k2 = np.random.rand(32, 3, 3, 64)
biais2 = np.random.rand(32)
k3 = np.random.rand(20, 3, 3, 32)
biais3 = np.random.rand(20)
weights_fcp = np.random.rand(180, 10)
biais_fcp = np.random.rand(10,)

conv1 = convolution(input_image, k1, biais1)
maxpool1 = MaxPool_v2(conv1)
conv2 = convolution(maxpool1, k2, biais2)
maxpool2 = MaxPool_v2(conv2)
conv3 = convolution(maxpool2, k3, biais3)
maxpool3 = MaxPool_v2(conv3)
reshape = reshape(maxpool3)
output = FCP(reshape, weights_fcp, biais_fcp)
print(output)
np.sum(output)
# PROBLEME AVEC EXPONENTIELLE... peut etre qu'avec les parametres du profs + en normalisant comme on devra le faire on aura pas ces problemes mais je ne suis pas sur

[nan nan nan nan nan nan nan nan nan nan]


/var/folders/h4/8j1m7rx96sb9bymrzdp6mp2m0000gp/T/ipykernel_50924/2365546596.py:10: RuntimeWarning: overflow encountered in exp
  sum += np.exp(x[i])
/var/folders/h4/8j1m7rx96sb9bymrzdp6mp2m0000gp/T/ipykernel_50924/2365546596.py:11: RuntimeWarning: overflow encountered in exp
  return np.exp(x) / sum
/var/folders/h4/8j1m7rx96sb9bymrzdp6mp2m0000gp/T/ipykernel_50924/2365546596.py:11: RuntimeWarning: invalid value encountered in divide
  return np.exp(x) / sum


nan

# Tests

In [ ]:
input_image = np.random.rand(3, 3, 20)
weights = np.random.rand(180, 10)
biais1 = np.random.rand(10,)

reshape = reshape(input_image)
output = FCP(reshape, weights, biais1)
print("Reshape ", reshape.shape)
print("output ", output.shape)
output

Reshape  (180,)
output  (10,)


array([1.85008817e-03, 1.72379814e-01, 7.56264326e-01, 4.95688493e-03,
       1.47238997e-02, 5.64001187e-03, 1.86226299e-02, 2.44207896e-02,
       5.42667803e-04, 5.98888706e-04])

In [191]:

matrice = np.random.randint(0, 256, size=(24, 24, 3))

R = matrice[:, :, 0]
G = matrice[:, :, 1]
B = matrice[:, :, 2]

# print("Canal Rouge (R):")
# print(R)
# print("\nCanal Vert (G):")
# print(G)
# print("\nCanal Bleu (B):")
# print(B)


In [192]:
m2=MaxPool(matrice)
R2 = m2[:, :, 0]
G2 = m2[:, :, 1]
B2 = m2[:, :, 2]

# print("Canal Rouge (R):")
# print(R2)
# print("\nCanal Vert (G):")
# print(G2)
# print("\nCanal Bleu (B):")
# print(B2)

output height: 11, output width: 11


In [193]:
input_image = np.random.rand(24, 24, 3)
k1 = np.random.rand(64, 3, 3, 3)  # 64 filtres de 3x3x3
biais1 = np.random.rand(64)

conv1_output = convolution(input_image, k1, biais1)
maxpool_output = MaxPool_v2(conv1_output)
print("Convolution 1 ", conv1_output.shape)  # (24, 24, 64)
print("Maxpool 1 ", maxpool_output.shape)  # (3, 3, 20)

Convolution 1  (24, 24, 64)
Maxpool 1  (12, 12, 64)
